In [1]:
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [4]:
base_url=r'https://www.google.com/maps'
text_to_search="Moradabad HR Consultancy"

In [9]:
driver= webdriver.Chrome()
driver.get(base_url)

input_box=driver.find_element(By.XPATH,'//*[@id="UGojuc"]')
input_box.send_keys(text_to_search)
input_box.send_keys(Keys.RETURN)
time.sleep(5)
lists=driver.find_element(By.XPATH,'//*[@id="149__uuXHif9"]')
time.sleep(5)

In [10]:
for card in lists.find_elements(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'):
    print(lists.text)

In [11]:
data=lists.text

In [12]:
# Open the file in write mode ('w') which automatically deletes existing content
with open("raw_scraped_data.txt", "w", encoding="utf-8") as file:
    file.write(data)

print("Old data deleted and new data successfully written to raw_scraped_data.txt")

Old data deleted and new data successfully written to raw_scraped_data.txt


In [13]:
data

'Results\n\ue88e\n\n\n\ue80d\nShare\nVG HR Professionals\n5.0(7)\nHuman resource consulting · 92, Stadium Rd\nOpen · Closes 6\u202fpm · 098977 90981\n\ue80b\nWebsite\n\ue52e\nDirections\n  "Fast response, professional guidance, and genuine job opportunities."\nPIONEER CONSULTANCY SERVICES\n4.7(416)\nRecruiter · Friends Colony Part 2, Near Wajid Nagar Complex, Prince Rd\nOpen · Closes 5\u202fpm · 073510 50570\n\ue80b\nWebsite\n\ue52e\nDirections\n  "They advise job according your interest, talent and skill."\nfavourable consulting pvt. ltd.\n4.9(34)\nRecruiter · RPJQ+RV3, Delhi Rd\nOpen · Closes 6\u202fpm\n\ue52e\nDirections\n  "Its a trusted and genuine company fir job placement in Mbd up fcpl"\nLAKSHYA A Placement & Training Division (LPD)\n4.9(17)\nRecruiter · Samrat ashok nagar, near galaxy banquet Kashiram gate the front of shri sai hospital\nOpen · Closes 6\u202fpm · 063965 81835\n\ue80b\nWebsite\n\ue52e\nDirections\n  "Simple process aur supportive guidance."\nNEOTECH CONSULTANCY

In [20]:
import re
import csv

# ==========================
# 1️⃣ READ TXT FILE
# ==========================
with open("raw_scraped_data.txt", "r", encoding="utf-8", errors="ignore") as f:
    raw_text = f.read()

# ==========================
# 2️⃣ CLEAN UNWANTED SYMBOLS
# ==========================
# remove strange icons
raw_text = re.sub(r'[\ue000-\uf8ff]', '', raw_text)

lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

# ==========================
# 3️⃣ EXTRACT DATA
# ==========================
records = []

i = 0
while i < len(lines) - 2:

    name = lines[i]

    # skip garbage lines
    if (
        name.lower().startswith(("results", "get the most", "sign in", "update results"))
        or "directions" in name.lower()
        or "website" in name.lower()
    ):
        i += 1
        continue

    # Look for category · address line
    if "·" not in lines[i + 2]:
        i += 1
        continue

    category_address = lines[i + 2]
    category, address = category_address.split("·", 1)
    category = category.strip()
    address = address.strip()

    phone = ""

    # Phone can appear in next few lines
    for j in range(i + 3, min(i + 6, len(lines))):
        match = re.search(r'(\+?\d[\d\s]{7,})', lines[j])
        if match:
            phone = match.group(1).strip()
            break

    records.append({
        "Name": name,
        "Category": category,
        "Address": address,
        "Phone": phone
    })

    i += 3

# ==========================
# 4️⃣ WRITE CSV
# ==========================
output_file = "Extracted_Data_map_scraper.csv"

with open(output_file, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["Name", "Category", "Address", "Phone"]
    )
    writer.writeheader()
    writer.writerows(records)

print(f"✅ CSV created successfully with {len(records)} rows")


✅ CSV created successfully with 29 rows
